In [1]:
import networkx as nx
import osmnx_func as osmfunc
import overpass
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
from shapely import length
import math
from pyproj import Transformer
import xml.etree.ElementTree as ET

In [2]:
api = overpass.API()

# fetch all ways and nodes
result = api.get("""
    area["name:en"="Copenhagen Municipality"] -> .a;
    (
    rel [type=route][route=subway][railway!=platform](area.a);
    node [station=subway](area.a);
    );
    (._;>>;);
    out geom;
    >;
    """, responseformat="xml")

tree = ET.ElementTree(ET.fromstring(result))

In [6]:
def check_continuity(start, stop, points):
    new_line = [start]
    points_to_sort = [[Point(start).distance(Point(stop)), stop]]
    
    #First sort by distance to origin
    for i in points:
        p = Point(i)
        points_to_sort.append([Point(start).distance(p), i])

    points_to_sort.sort(key=lambda x: x[0])

    points_to_sort = [i[1] for i in points_to_sort]

    while points_to_sort != []:
        previous_point = Point(new_line[-1])
        closest = 1_000_000
        next_point = None
        for i in points_to_sort:
            cal_point = Point(i)
            dist = previous_point.distance(cal_point)
            if dist < closest:
                closest = dist
                next_point = i
        
        new_line.append(next_point)
        
        if next_point == stop:
            break
        
        points_to_sort.remove(next_point)
        
    #new_line.append(stop)
    return new_line

In [7]:
data = []

for relation in tree.findall('relation'):
    relation_id = relation.attrib['id']
    stations_node_ids = []
    points_on_line = []
    print('###################')
    #Get members of relations
    for mem in relation.findall('member'):
        #Get node ids
        if mem.attrib['type'] == 'node':
            stations_node_ids.append(mem.attrib['ref'])
        #Get way's corrdinates
        elif mem.attrib['type'] == 'way':
            #Get points on line:
            for point in mem.findall('nd'):
                points_on_line.append((float(point.attrib['lon']), float(point.attrib['lat'])))
        else:
            pass
    
    from_name = None
    from_coords = points_on_line[0]
    
    for sta in stations_node_ids:
        #Get stations data and point
        station_data = nodes[nodes['id'] == sta]
        station_point = (station_data['lon'].item(), station_data['lat'].item())
        to = station_data.name.item()

        #Points to make linestring
        #Get station index for the line
        if station_point in points_on_line:
            index = points_on_line.index(station_point)
            points = check_continuity(start = from_coords, stop = station_point, points = points_on_line[:index])
        else:
            points = check_continuity(start = from_coords, stop = station_point, points = points_on_line)
            
        #Make linestring
        line_geo = LineString(points)
        
        #Add edge
        to = station_data.name.item()
        print(f"{from_name} -> {to}")
        data.append([to, None, Point(station_point)])
        data.append([from_name, to, line_geo])

        #Update variables for next iter.
        from_name = to
        from_idx = index
        from_coords = station_point
        #remove used points
        for i in points[1:-1]:
            if i in points_on_line:
                points_on_line.remove(i)


data = gpd.GeoDataFrame(data, columns = ['from', 'to', 'geometry'])
data = data[~data['from'].isna()]
data.explore()

###################
None -> Vanløse
Vanløse -> Flintholm
Flintholm -> Lindevang
Lindevang -> Fasanvej
Fasanvej -> Frederiksberg
Frederiksberg -> Forum
Forum -> Nørreport
Nørreport -> Kongens Nytorv
Kongens Nytorv -> Christianshavn
Christianshavn -> Islands Brygge
Islands Brygge -> DR Byen
DR Byen -> Sundby
Sundby -> Bella Center
Bella Center -> Ørestad
Ørestad -> Vestamager
###################
None -> Vanløse
Vanløse -> Flintholm
Flintholm -> Lindevang
Lindevang -> Fasanvej
Fasanvej -> Frederiksberg
Frederiksberg -> Forum
Forum -> Nørreport
Nørreport -> Kongens Nytorv
Kongens Nytorv -> Christianshavn
Christianshavn -> Amagerbro
Amagerbro -> Lergravsparken
Lergravsparken -> Øresund
Øresund -> Amager Strand
Amager Strand -> Femøren
Femøren -> Kastrup
Kastrup -> Lufthavnen (Metro)
###################
None -> Vestamager
Vestamager -> Ørestad
Ørestad -> Bella Center
Bella Center -> Sundby
Sundby -> DR Byen
DR Byen -> Islands Brygge
Islands Brygge -> Christianshavn
Christianshavn -> Kongens 

# Construct a graph

In [4]:
import networkx as nx

def get_meta_from_tree(tree, osm_type):
    """
    Get the meta data from nodes and ways
    in the element tree, returns a list
    of dicts with the meta_data
    """
    dicts = []
    for element in tree.findall(osm_type):
        tags = element.findall('tag')
        if tags != []:
            temp_dict = {}
            temp_dict['id'] = int(element.get('id'))
            temp_dict['osm_type'] = osm_type
            if osm_type == 'node':
                temp_dict['lat'] = float(element.get('lat'))
                temp_dict['lon'] = float(element.get('lon'))
            for tag in element.findall('tag'):
                temp_dict[tag.get('k')] = tag.get('v')
            dicts.append(temp_dict)
    return dicts

def get_nodes(tree):
    """
    Get all the stations of a relation,
    returns a dict with list, with a 
    stations (point, osm_id)
    """
    node_order = {} #key = rel_id, value = stations nodes
    for rel in tree.findall('relation'):
        nodes = []
        relation_id = int(rel.attrib['id'])

        #Get members of relations
        for mem in rel.findall('member'):
            #Get node ids
            if mem.attrib['type'] == 'node':
                lon = float(mem.attrib['lon'])
                lat = float(mem.attrib['lat'])
                nodes.append([(lon,lat), int(mem.attrib['ref'])])
        node_order[relation_id] = nodes
    return node_order

def get_way_order(tree):
    """
    Get all the ways of a relation,
    creates a network, where each node 
    is a point of the ways.
    Returns a dict with a nx.Graph of the
    ways.
    """
    rel_graph_dict = {} 
    for rel in tree.findall('relation'):
        relation_id = int(rel.attrib['id'])
        G = nx.Graph()
        #Get members of relations
        for mem in rel.findall('member'):
            #Check if it is a way
            if mem.attrib['type'] == 'way':
                osm_id = int(mem.attrib['ref'])
                previous_point = None

                #Add edge in the graph
                for point in mem.findall('nd'):
                    lon = float(point.attrib['lon'])
                    lat = float(point.attrib['lat'])
                    if previous_point == None:
                        previous_point = (lon, lat)
                    else:
                        G.add_edge(u_of_edge = previous_point, v_of_edge = (lon, lat), attr={'osm_id':osm_id})
                        #print(f"{previous_point} -> {(lon, lat)}")
                        previous_point = (lon, lat)
        rel_graph_dict[relation_id] = G
    
    return rel_graph_dict

def check_stations(node_order, way_graphs):
    """
    Checks if the stations are in the graph, if 
    not it match them to the point on the line
    closest to the station
    """
    new_node_order = {}
    for rel_id in way_graphs.keys():
        nodes = node_order[rel_id]
        G = way_graphs[rel_id]
        graph_nodes = list(G.nodes())
        nodes_in_graph = []
        for n in nodes:
            if G.has_node(n):
                nodes_in_graph.append(n)       
            else: #Match to point on the ways
                closest = 1_000_000
                close = None
                p = Point(n[0][0], n[0][1])
                for i in graph_nodes:
                    dist = p.distance(Point(i[0], i[1]))
                    if dist < closest:
                        closest = dist
                        close = i
                if close != None:
                    nodes_in_graph.append((close, n[1]))
        new_node_order[rel_id] = nodes_in_graph
    return new_node_order

def get_meta(tree):
    nodes_meta = get_meta_from_tree(tree = tree, osm_type='node')
    way_meta = get_meta_from_tree(tree = tree, osm_type='way')
    meta_data = pd.DataFrame(nodes_meta+way_meta)
    meta_data = meta_data.drop_duplicates(['id', 'osm_type'])
    meta_data.index = meta_data.id
    meta_data = meta_data.to_dict(orient = 'index')
    return meta_data

def clean_meta_dict(d, keep_coord = True):
    """
    Takes dict and removes nan values
    """
    clean_meta = d.copy()
    for key,value in d.items():
        if key == 'lat' and keep_coord:
            del clean_meta[key]
            clean_meta['y'] = value
        elif key == 'lon'and keep_coord:
            del clean_meta[key]
            clean_meta['x'] = value
        elif type(value) != str and math.isnan(value):
            del clean_meta[key]
        else:
            None 
    return clean_meta

def get_osmid_from_shortest_path(G, path):
    ids = []
    for idx in range(1, len(path)-2): #to exclude the stations
        att = G[path[idx]][path[idx+1]]['attr']
        ids.append(att['osm_id'])
    return list(set(ids))

def line_lenght(line):
    # Transformer to convert from WGS84 to EPSG:3857 (meters)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    projected_line = LineString([transformer.transform(*coord) for coord in line.coords])
    length_in_meters = projected_line.length
    return length_in_meters

def create_network(tree):
    node_order = get_nodes(tree)
    way_graphs = get_way_order(tree)
    meta_data = get_meta(tree)

    node_order = check_stations(node_order, way_graphs)

    great_graph = nx.MultiDiGraph()
    #Plot graph
    for rel_id in way_graphs.keys():
        G = way_graphs[rel_id]
        nodes = node_order[rel_id]
        for n_idx in range(len(nodes)-1):
            u = nodes[n_idx]
            u_meta = clean_meta_dict(meta_data[u[1]])

            v = nodes[n_idx+1]
            v_meta = clean_meta_dict(meta_data[v[1]])

            great_graph.add_nodes_from([(u[1], u_meta)])
            great_graph.add_nodes_from([(v[1], v_meta)])
            
            try:
                shortest_path = nx.shortest_path(G, u[0], v[0])
                osm_ids = get_osmid_from_shortest_path(G, shortest_path)
                attr_dict = {}
                for i in osm_ids:
                    attr_dict[int(i)] = clean_meta_dict(meta_data[i], keep_coord= False)
                    del attr_dict[i]['id']
                line = LineString(shortest_path)
                great_graph.add_edge(u_for_edge = u[1], v_for_edge = v[1], osmid = osm_ids, geometry = line, 
                                                                                  length= line_lenght(line),
                                                                                  attr_dict = attr_dict
                                                                                  )
            except:
                ...
        
    return great_graph

In [5]:
network = create_network(tree = tree)

In [6]:
osmfunc.city_to_files(G = network, city = 'Copenhagen', osm_type = 'Subway', nx_type = 'multidigraph')